In [ ]:
#make sure you have the latest version of the weaviate-client installed
!pip install weaviate-client -U
!pip intall PILLOW

In [ ]:
#python3 -m venv venv
from PIL import Image, ImageDraw  
import base64, os, weaviate
from io import BytesIO
from pathlib import Path

In [ ]:
client = weaviate.connect_to_local()
print("Connected to Weaviate")

In [ ]:
def base64_to_image(image_data_base64):
    image_data = base64.b64decode(image_data_base64)
    image = Image.open(BytesIO(image_data))
    return image


def file_to_base64(path):
    with open(path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')



def create_schema(collectionName):
    import weaviate.classes as wvc
    from weaviate.classes.config import Property, DataType

    if(client.collections.exists(collectionName)):
        client.collections.delete(collectionName)

    client.collections.create(
        name=collectionName,
        properties=[
            Property(name="image", data_type=DataType.BLOB),
            Property(name="x_coordinate", data_type=DataType.INT),
            Property(name="y_coordinate", data_type=DataType.INT)
        ],
        vectorizer_config=wvc.config.Configure.Vectorizer.multi2vec_clip(
            image_fields=["image"] 
        )
    )
    
    print("Collection created")


def split_image_locally(input_image_path, split_width, split_height, output_folder):
    image = Image.open(input_image_path)
    image_width, image_height = image.size

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for y in range(0, image_height, split_height):
        for x in range(0, image_width, split_width):
            bbox = (x, y, x + split_width, y + split_height)
            split_image = image.crop(bbox)
            filename = f"{y // split_height + 1}_{x // split_width + 1}_waldo.png"
            split_image.save(os.path.join(output_folder, filename))

def upload_to_weaviate(input_image_folder,collectionName):
    collection = client.collections.get(collectionName)
    filelist = Path(input_image_folder).glob('*.png')
    filelist = sorted([x.name for x in filelist])
    #loop through all images in the input_image_folder and upload them to weaviate
    for filename in filelist:
        if filename.endswith(".png"):
            image = Image.open(os.path.join(input_image_folder, filename))
            image_data = image_to_base64(image)
            x, y = map(int, filename.split('_')[:2])
            obj = {"image": image_data, "x_coordinate": x, "y_coordinate": y}
            collection.data.insert(obj)
            print(f'Added coordinates x:{x} y:{y} to Weaviate')
            #note - not using batching here so we know when it completes each record

def findWaldo(waldo_image,split_width,split_height,collectionName):
    print("preparing waldo image to search for")
    collection = client.collections.get(collectionName)
    resized_waldo = 'images/waldo_resized.png'
    waldo_image = Image.open(waldo_image)
    #shrink and scale the image down the defined block size
    waldo_image = waldo_image.resize((split_width, split_height))
    print("image resized")
    #temporarily save the image to a file
    waldo_image.save(resized_waldo)
    searchableWaldo = image_to_base64(waldo_image)
    #print(searchableWaldo)
    print("image converted to base64")

    #perform a near image search to find the closest match to the waldo image
    response = collection.query.near_image(
        near_image=searchableWaldo,
        return_properties=['x_coordinate','y_coordinate'],
        limit=1
    )
    
    coordinates = []
    for obj in response.objects:
        print(obj.properties)
        coordinates.append((obj.properties['x_coordinate'], obj.properties['y_coordinate']))

    
    print(coordinates)
    return coordinates

def reassemble_and_highlight(original_image_path, split_width, split_height, highlight_rows_columns, output_folder):
    original_image = Image.open(original_image_path)
    total_width, total_height = original_image.size
    
    print(total_width, total_height)

    new_image = Image.new('RGB', (total_width, total_height))

    # Loop over the folder with the split images and reassemble the original image
    for filename in os.listdir(output_folder):
        if filename.endswith("_waldo.png"):  # Ensuring only the intended images are processed
            # Extract row and column numbers from the filename
            part_row, part_col = map(int, filename.split('_')[:2])
            # Calculate the position where this part should be pasted in the new image
            position = ((part_col - 1) * split_width, (part_row - 1) * split_height)
            # Open the image tile and paste it into the new image
            image_part = Image.open(os.path.join(output_folder, filename))
            new_image.paste(image_part, position)

            # Highlight the tile if it is part of the highlight list
            if (part_row, part_col) in highlight_rows_columns:
                draw = ImageDraw.Draw(new_image)
                outline_width = 8  # highlight thickness
                draw.rectangle(
                    [position[0], position[1], position[0] + split_width, position[1] + split_height],
                    outline="red", width=outline_width
                )

    return new_image

In [ ]:
input_image_path = 'images/waldo_ski_image.jpeg'  # Change this to the path of your input image
collectionName='WaldoCollection' #name of the collection to create in weaviate
split_width = 250  # Width of each tile image
split_height = 250  # Height of each tile image

In [ ]:
create_schema(collectionName)  # Only run this once to create the schema, it will delete if it already exists

In [ ]:
split_image_locally(input_image_path, split_width, split_height, 'images/output')
print("Image is now split into local directory")

In [ ]:
upload_to_weaviate('images/output',collectionName)
print("Images are now uploaded to Weaviate")

In [ ]:
input_image_path = 'images/waldo_ski_image.jpeg'
query_image = 'images/Waldo_ski.png'

highlighted_parts = findWaldo(query_image,split_width,split_height,collectionName)
print(highlighted_parts)


if highlighted_parts == []:
    print("Waldo not found")
else:
   print("Waldo found!")
   reassembled_image = reassemble_and_highlight(input_image_path, split_width, split_height, highlighted_parts, 'images/output')
   reassembled_image.show()  # This will display the reassembled image with highlighted parts